# 1Hive Economy  

This model aims to create a simulation of the 1hive economy, it assumes that 1hive can capture fee inflows by directing outflows towards productive activities and represents this growth potential as a logistic curve that represents market saturation. We assume that outflows will result in increased market saturation, and that participants will increase outflows when the perceived opportunity for growth is higher, and reduce outflows as the opportunity for growth decreases. 
    
For the purpose of this model we differentiate the honey supply into two buckets, the common pool and the circulating supply. We model inflows to the system (in stable value terms) as a function of saturation and market size, and have saturation decay over time. We define a conversion price that adjusts each timestep as a ratio of market cap to inflows divided by saturation, and use this price to model inflows as a movement of honey from the circulating supply into the common pool. 

We set an outflow boundary from the common pool as an abstraction of how conviction voting limits the **maximum possible outflow rate from a pool of funds given a certain parameterization of conviction voting**, though in practice the real outflow rate would tend to be much lower than this boundary due to the ability for participants to support signaling proposals (eg abstain), or for proposal to recieve some support but ultimately fail to reach sufficient support to execute. To reflect the expected behavior of participants to modulate outflows based on their perception of the growth potential we make the actual outflow rate adjust based on level of saturation. 

We are interested in exploring parameter choices for internal mechanisms and analzying the sensitivity to parameters of environmental processes to better understand how the protocol will respond in terms of price, supply, and common pool reserves across various stages of growth. 




In [1]:
import random as rand
import numpy as np 

In [2]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
genesis_states = {
    'reserve': 7473, # Current common pool balance
    'circulation': 18765, # Total honey supply minus common pool balance (26238 - 7473)
    'size': 100000, # Total market size in terms of fiat inflows per month  
    'saturation': 0.0, # Percent of market captured by 1hive
    'production': 0.0, # Production state accounting for outflows and upkeep
    'utility': 0.0, # representing diminishing marginal returns on production
    'price': 10, # modeled as a random walk biased towards a target price based on size, saturation, circulation, and valuation ratio parameter. 
    'target_price':1, # Tracks the implied valuation based on our assumptions and influeces the stochastic spot price. 
    'netflow': 0.0,
    'adjustment': 0.0
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Model Params
We define some parameters which can be used to tune the behavior of issuance and distribution in the model. Paramters assume that each time step of the model relfect 1 month of real time, the simulation will run for 120 timesteps giving the model as a whole a 10 year time horizon.  

In [3]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# List of all the state variables in the system and their initial values
params = {
    'throttle': [0.008], # maximimum proportion of the total supply that can be adjusted in each timestep 
    'outflow_boundary': [0.05], # max proportion of common pool funds that can be spent each timestep 
    'target_reserve_ratio': [0.2], # target ratio of common pool funds to total supply
    'valuation_ratio': [2], # ratio of price to inflows (monthly) when at market saturation 
    'productivity': [1.5],  # scalar parameter to determine impact of outflows on market saturation
    'maintainence': [0.1], # scalar parameter to determine magnitude of saturation decays each time step
    'growth': [0], # rate at which the market size will grow each time step 
    'p_variance': [.25], # variance of price changes
    'p_pull': [.6] # odds of moving towards or away from price target, a value of .5 is an unbiased random walk. 
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Timestep
We assume that each timestep in cadcad represents 1 month in real time, scaling model parameters to reflect that timescale. 

# Policies and State Update Functions

We define a supply policy and four state update fuctions. 


In [4]:
def supply_policy(params, step, sH, s):

    # first we calculate outlflow as a function of the reserve, outflow boundary, and saturation
    outflow = s['reserve'] * params['outflow_boundary'] / (s['saturation'] + 1)
    
    # then we calculate inflow as a function size, saturation, price
    inflow = s['size'] * s['saturation'] / s['price']
    
    netflow = inflow - outflow

    # then we calculate current state
    reserve = s['reserve'] + netflow 
    circulation = s['circulation'] - netflow
    supply = reserve + circulation
    ratio = reserve / supply
    

    # Proportional control https://en.wikipedia.org/wiki/Proportional_control
    # corrections are made proportionally to the difference between the target and the current value

    # e = (params['target_reserve_ratio'] - ratio) / 12
    e = (1 - ratio / params['target_reserve_ratio'])/12

    if params['throttle'] != 0: 
        # Corrections bounded by a maximum issuance rate parameter 
        if e < 0:
            adjustment = max(e, -params['throttle']) * supply
        else:
            adjustment = min(e, params['throttle']) * supply 
    else:
        # Corrections are unbounded, issuance is bounded by the maximum outflow rate 
        adjustment = e * supply 


    return ({'netflow':netflow, 'adjustment':adjustment})


def saturation_process(params, step, sH, s):

    # outflows are a function of outflow_boundary parameter and current saturation
    # as saturation approaches 1, outflow_boundary is reduced by half. 
    outflow = s['reserve'] * params['outflow_boundary'] / (1 + s['saturation'] ** 2 )

    # production reduced by the maintainance parameter and then increased by the impact of outflows, which depends on the productivity paramater and the current price. 
    production = s['production'] * (1 - params['maintainence']) + outflow * params['productivity'] * s['price']

    # Utility represents the diminishing returns to production and is bounded at 10, we use the size value to shape the curve because it relates to the maximum inflows and steady state outflows. 
    utility = 10 * production / ( production + s['size'] )
    # utility = log

    # saturation modeled as a logistic function, shifted by 6 so that a 0 utility means near 0 saturation, a utility of 5 is around the inflection point, and 10 is near 1. 
    saturation = 1 / (1 + np.exp(-utility + 6))

    # size is a function of current size and growth rate
    size = s['size'] * (1 + params['growth'])


    return ({ 'saturation':saturation, 'size':size, 'utility':utility, 'production':production})

def price_policy(params, step, sH, s):
    # price follows a random walk that is pulled towards a target price. 

    # First we calculate the target price as a function of the size, valuation ratio, and state of the supply
    potential_inflows_per_token = s['size'] / (s['circulation'] + s['reserve'])
    # valuation ratio decreases as system reaches market saturation 
    adjusted_valuation_ratio = params['valuation_ratio'] / max(0.5, s['saturation'])
    # Price increases exponentially as the ratio of circulating supply to total supply decreases
    supply_sensitivity = 1 / (s['circulation'] / (s['circulation'] + s['reserve']) ) ** 2 

    target_price = potential_inflows_per_token * adjusted_valuation_ratio * supply_sensitivity

    # Price adjusts from previous timestep as a random walk, with a probability of moving closer to the price target determined by a parameter p_pull. 
    
    if np.random.random() < params['p_pull']: # toward target
        if s['price'] < target_price:
            direction = 1
        else:
            direction = -1 
    else: # away from target
        if s['price'] < target_price:
            direction = -1 
        else:
            direction = 1

    magnitude = np.abs(np.random.normal(0,params['p_variance']))

    price = s['price'] + s['price'] * direction * magnitude * supply_sensitivity

    return ({'price':price, 'target_price':target_price})

def update_reserve(params, step, sH, s, _input):
    key = 'reserve'
    value = s['reserve'] + _input['netflow'] + _input['adjustment']
    return (key, value)

def update_utility(params, step, sH, s, _input):
    key = 'utility'
    value = _input['utility']
    return (key, value)

def update_production(params, step, sH, s, _input):
    key = 'production'
    value = _input['production']
    return (key, value)

def update_circulation(params, step, sH, s, _input):
    key = 'circulation'
    value = s['circulation'] - _input['netflow']
    return (key, value)

def update_size(params, step, sH, s, _input):
    key = 'size'
    value =  _input['size']
    return (key, value)

def update_saturation(params, step, sH, s, _input):
    key = 'saturation'
    value =  _input['saturation']
    return (key, value)

def update_price(params, step, sH, s, _input):
    key = 'price'
    value = _input['price']
    return (key, value)

def update_target_price(params, step, sH, s, _input):
    key = 'target_price'
    value = _input['target_price']
    return (key, value)

def update_netflow(params, step, sH, s, _input):
    key = 'netflow'
    value =  _input['netflow']
    return (key, value)

def update_adjustment(params, step, sH, s, _input):
    key = 'adjustment'
    value =  _input['adjustment']
    return (key, value)

# Partial State Update Blocks


In [5]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# In the Partial State Update Blocks, the user specifies if state update functions will be run in series or in parallel
partial_state_update_blocks = [
    { 
        'policies': {
            'supply_policy': supply_policy,
            'saturation_process': saturation_process,
            'price_policy': price_policy

        },
        'variables': { # The following state variables will be updated simultaneously
            'reserve': update_reserve,
            'circulation': update_circulation,
            'netflow': update_netflow,
            'adjustment': update_adjustment,
            'price': update_price,
            'target_price': update_target_price,
            'saturation': update_saturation,
            'utility': update_utility,
            'size': update_size,
            'production': update_production
        }
    }
]
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Simulation Configuration Parameters
Lastly, we define the number of timesteps and the number of Monte Carlo runs of the simulation. These parameters must be passed in a dictionary, in `dict_keys` `T` and `N`, respectively. In our example, we'll run the simulation for 10 timesteps. And because we are dealing with a deterministic system, it makes no sense to have multiple Monte Carlo runs, so we set `N=1`. We'll ignore the `M` key for now and set it to an empty `dict`

In [6]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# Settings of general simulation parameters, unrelated to the system itself
# `T` is a range with the number of discrete units of time the simulation will run for;
# `N` is the number of times the simulation will be run (Monte Carlo runs)
# In this example, we'll run the simulation once (N=1) and its duration will be of 10 timesteps
# We'll cover the `M` key in a future article. For now, let's omit it
sim_config_dict = {
    'T': range(240),
    'N': 1,
    'M': params
}
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

# Putting it all together
We have defined the state variables of our system and their initial conditions, as well as the state update functions, which have been grouped in a single state update block. We have also specified the parameters of the simulation (number of timesteps and runs). We are now ready to put all those pieces together in a `Configuration` object.

In [7]:
#imported some addition utilities to help with configuration set-up
from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD import configs

exp = Experiment()
c = config_sim(sim_config_dict)
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
# The configurations above are then packaged into a `Configuration` object
del configs[:]
exp.append_configs(initial_state=genesis_states, #dict containing variable names and initial values
                       partial_state_update_blocks=partial_state_update_blocks, #dict containing state update functions
                       sim_configs=c #preprocessed dictionaries containing simulation parameters
                      )

# Running the engine
We are now ready to run the engine with the configuration defined above. Instantiate an ExecutionMode, an ExecutionContext and an Executor objects, passing the Configuration object to the latter. Then run the `execute()` method of the Executor object, which returns the results of the experiment in the first element of a tuple.

In [8]:
%%capture
from cadCAD.engine import ExecutionMode, ExecutionContext
exec_mode = ExecutionMode()
local_mode_ctx = ExecutionContext(exec_mode.local_mode)

from cadCAD.engine import Executor

simulation = Executor(exec_context=local_mode_ctx, configs=configs) # Pass the configuration object inside an array
raw_system_events, tensor_field, sessions = simulation.execute() # The `execute()` method returns a tuple; its first elements contains the raw results


# Analyzing the results
We can now convert the raw results into a DataFrame for analysis

In [9]:
%matplotlib inline
import pandas as pd
simulation_result = pd.DataFrame(raw_system_events)
simulation_result['total_supply'] = simulation_result['reserve'] + simulation_result['circulation']
simulation_result['market_cap'] = simulation_result['total_supply'] * simulation_result['price']
simulation_result['ratio'] = simulation_result['reserve'] / simulation_result['total_supply']
simulation_result.set_index(['subset', 'run', 'timestep', 'substep'])

reserve   circulation    size  saturation  \
subset run timestep substep                                                  
0      1   0        0        7473.000000  18765.000000  100000    0.000000   
           1        1        6889.446000  19138.650000  100000    0.004197   
           2        1        6368.350754  19451.520478  100000    0.007318   
           3        1        5914.126985  19699.185277  100000    0.009834   
           4        1        5492.294423  19916.111341  100000    0.013179   
...                                  ...           ...     ...         ...   
           236      1        3712.508175    478.755014  100000    0.970166   
           237      1        3625.638484    532.094600  100000    0.971276   
           238      1        3539.095590    585.375628  100000    0.972173   
           239      1        3502.299743    589.175706  100000    0.971841   
           240      1        3455.838857    602.904788  100000    0.971647   

                               production   utility        price  \
subset run timestep substep                                        
0      1   0        0        0.000000e+00  0.000000    10.000000   
           1        1        5.604750e+03  0.530729    13.913478   
           2        1        1.223336e+04  1.089993    10.693387   
           3        1        1.611719e+04  1.388011    12.956918   
           4        1        2.025208e+04  1.684136    17.641234   
...                                   ...       ...          ...   
           236      1        1.829779e+06  9.481806  2373.285594   
           237      1        1.987212e+06  9.520892  2511.012420   
           238      1        2.139839e+06  9.553539  1131.411633   
           239      1        2.080248e+06  9.541337  1294.428972   
           240      1        2.047083e+06  9.534252  1382.680345   

                             target_price     netflow  adjustment  simulation  \
subset run timestep substep                                                     
0      1   0        0            1.000000    0.000000    0.000000           0   
           1        1           29.805306 -373.650000 -209.904000           0   
           2        1           28.423646 -312.870478 -208.224768           0   
           3        1           27.296499 -247.664799 -206.558970           0   
           4        1           26.401537 -216.926064 -204.906498           0   
...                                   ...         ...         ...         ...   
           236      1         4937.465361  -58.338236  -33.800510           0   
           237      1         3769.661646  -53.339586  -33.530106           0   
           238      1         3023.890508  -53.281029  -33.261865           0   
           239      1         2476.197581   -3.800077  -32.995770           0   
           240      1         2425.632080  -13.729083  -32.731804           0   

                             total_supply    market_cap     ratio  
subset run timestep substep                                        
0      1   0        0        26238.000000  2.623800e+05  0.284816  
           1        1        26028.096000  3.621413e+05  0.264693  
           2        1        25819.871232  2.761019e+05  0.246645  
           3        1        25613.312262  3.318696e+05  0.230901  
           4        1        25408.405764  4.482356e+05  0.216161  
...                                   ...           ...       ...  
           236      1         4191.263189  9.947065e+06  0.885773  
           237      1         4157.733083  1.044012e+07  0.872023  
           238      1         4124.471219  4.666475e+06  0.858073  
           239      1         4091.475449  5.296124e+06  0.855999  
           240      1         4058.743645  5.611945e+06  0.851455  

[241 rows x 14 columns]

# Analysis 

After adding in concepts for market saturation, productivity and diminishing marginal returns of outflows, and a pricing function we can see how the system evolves with various paramater choices.  

With sufficient **productivity** relative to maintainence, the system stabilizes near the top of the logistic curve representing market saturation. We see market cap increase while saturating the market, then stabilize and remain steady. Price increases faster during the growth phases, but continues to steadily increases after reaching market saturation as the circulating supply decreases as a result of steady inflows. 


In [10]:



import plotly.express as px
fig = px.line(
    simulation_result,
    x='timestep',
    y=['total_supply', 'circulation', 'reserve'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [11]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['netflow', 'adjustment'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [12]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['ratio', 'saturation'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [13]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['price', 'target_price'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [14]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['market_cap', 'size'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()

In [15]:
fig = px.line(
    simulation_result,
    x='timestep',
    y=['utility'],
    facet_row='subset',
    height=800,
    template='seaborn'
)

fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)

fig.show()